In [ ]:
from siphon.catalog import TDSCatalog
from pathlib import Path
from google.cloud import storage
import io, os
import fsspec
import urllib.request
from urllib.parse import urlparse
from pathlib import Path
import time
from dotenv import load_dotenv

In [ ]:
load_dotenv(os.path.expanduser('~/.cefi_env'))

In [ ]:
bucket_name = 'noaa-oar-cefi-regional-mom6'
project_id = os.getenv("GOOGLE_PROJECT_ID")
storage_client = storage.Client.from_service_account_json(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))
bucket = storage_client.get_bucket(bucket_name)

In [ ]:
dirs = [
    'northwest_atlantic/full_domain/hindcast/daily/raw/r20230520/',
    'northwest_atlantic/full_domain/hindcast/daily/regrid/r20230520/',
    'northwest_atlantic/full_domain/hindcast/monthly/raw/r20230520/',
    'northwest_atlantic/full_domain/hindcast/monthly/regrid/r20230520/',
    'northwest_atlantic/full_domain/seasonal_forecast/monthly/raw/r20250212/',
    'northwest_atlantic/full_domain/seasonal_forecast/monthly/regrid/r20250212/',
    'northwest_atlantic/full_domain/seasonal_reforecast/monthly/raw/r20250212/',
    'northwest_atlantic/full_domain/seasonal_reforecast/monthly/regrid/r20250212/',
]
# dirs = [
#     'northeast_pacific/full_domain/hindcast/daily/raw/r20241015/',
#     'northeast_pacific/full_domain/hindcast/daily/regrid/r20241015/',
#     'northeast_pacific/full_domain/hindcast/monthly/raw/r20241015/',
#     'northeast_pacific/full_domain/hindcast/monthly/regrid/r20241015/'
# ]

In [ ]:
# N.B. These files are small so I don't check whether they exist, I just copy them which will replace what's there.
for dir in dirs:
    fs_read = fsspec.filesystem('file')
    flist = fs_read.glob(f'{dir}all.json')
    for outfile in flist:
        destination_blob_name = outfile[outfile.find('northwest'):]
        blob = bucket.blob(destination_blob_name)
        blob.upload_from_filename(outfile, content_type='application/json')